In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import pytorch_lightning as pl
import wandb
from pytorch_lightning.loggers import WandbLogger
import pickle
import json

from DataLoader import VideoQADataLoader

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!wandb login 3ed7a1bc59fad48beeadc999df34dbee428be831

wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


In [4]:
wandb.init(project="video-qa-hcrn-recvis")

wandb: Currently logged in as: nicolas-dufour (use `wandb login --relogin` to force relogin)


## Load Data

In [5]:
def invert_dict(d):
    return {v: k for k, v in d.items()}
class VideoQADataModule(pl.LightningDataModule):
    def __init__(self, data_path,dataset_name,batch_size,text_embedding_method):
        super().__init__()
        
        self.batch_size = batch_size
        self.dataset_name = dataset_name
        if(self.dataset_name == 'msvd-qa' or self.dataset_name == 'msrvtt-qa'):
            self.question_type = 'none'
        elif(self.dataset_name == 'tgif-qa_frameqa'):
            self.question_type = 'frameqa'
        self.text_embedding_method = text_embedding_method
        self.num_workers = 4
        
        self.dataset_path = f"{data_path}/{self.dataset_name}"
        
        if(self.text_embedding_method == 'glove'):
            with open(f"{self.dataset_path}/{self.text_embedding_method}_question_embedding/{self.dataset_name}_train_questions.pt", 'rb') as f:
                obj = pickle.load(f)
                glove_matrix = obj['glove']
            self.glove_matrix = glove_matrix
        with open(f"{self.dataset_path}/{self.dataset_name}_vocab_{self.text_embedding_method}.json", 'r') as f:
            vocab = json.load(f)
            vocab['question_idx_to_token'] = invert_dict(vocab['question_token_to_idx'])
            vocab['answer_idx_to_token'] = invert_dict(vocab['answer_token_to_idx'])
            vocab['question_answer_idx_to_token'] = invert_dict(vocab['question_answer_token_to_idx'])
        self.vocab = vocab

    def train_dataloader(self):
        return VideoQADataLoader(
                question_type = self.question_type,
                batch_size = self.batch_size,
                num_workers = self.num_workers,
                question_pt = f"{self.dataset_path}/{self.text_embedding_method}_question_embedding/{self.dataset_name}_train_questions.pt" ,
                appearance_feat = f"{self.dataset_path}/{self.dataset_name}_appearance_feat.h5",
                motion_feat = f"{self.dataset_path}/{self.dataset_name}_motion_feat.h5",
                shuffle=True
        )

    def val_dataloader(self):
        return VideoQADataLoader(
                question_type = self.question_type,
                batch_size = self.batch_size,
                num_workers = self.num_workers,
                question_pt = f"{self.dataset_path}/{self.text_embedding_method}_question_embedding/{self.dataset_name}_val_questions.pt" ,
                appearance_feat = f"{self.dataset_path}/{self.dataset_name}_appearance_feat.h5",
                motion_feat = f"{self.dataset_path}/{self.dataset_name}_motion_feat.h5",
                shuffle=False
        )

    def test_dataloader(self):
        return VideoQADataLoader(
                question_type = self.question_type,
                batch_size = self.batch_size,
                num_workers = self.num_workers,
                question_pt = f"{self.dataset_path}/{self.text_embedding_method}_question_embedding/{self.dataset_name}_test_questions.pt" ,
                appearance_feat = f"{self.dataset_path}/{self.dataset_name}_appearance_feat.h5",
                motion_feat = f"{self.dataset_path}/{self.dataset_name}_motion_feat.h5",
                shuffle=False
        )

In [6]:
msvd_glove_data_module = VideoQADataModule('data','msvd-qa',batch_size=32,text_embedding_method='glove')
msrvtt_glove_data_module = VideoQADataModule('data','msrvtt-qa',batch_size=32,text_embedding_method='glove')
tgif_glove_data_module = VideoQADataModule('data','tgif-qa_frameqa',batch_size=32,text_embedding_method='glove')

## Defining the base model

In [7]:
import model.HCRN as HCRN

In [8]:
class HCRN_glove(pl.LightningModule):
    def __init__(self,glove_matrix,lr,model_kwargs,optimizer = 'AdamW'):
        super().__init__()
        
        self.lr = lr
        self.optimizer = optimizer
        self.criterion = nn.CrossEntropyLoss()
        
        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()
        
        glove_matrix = torch.FloatTensor(glove_matrix)
        self.model = HCRN.HCRNNetwork(**model_kwargs)
        with torch.no_grad():
            self.model.linguistic_input_unit.encoder_embed.weight.set_(glove_matrix)
    
    def forward(self,ans_candidates, ans_candidates_len, video_appearance_feat, video_motion_feat, question,
                question_len):
        return self.model(ans_candidates, ans_candidates_len, video_appearance_feat, video_motion_feat, question,
                question_len)
    
    def configure_optimizers(self):
        if(self.optimizer == 'Adam'):
            optimizer = optim.Adam(self.parameters(), lr=self.lr)
        elif(self.optimizer == 'AdamW'):
            optimizer = optim.AdamW(self.parameters(), lr=self.lr)
        else:
            raise "Optimizer not supported"
        scheduler = optim.lr_scheduler.StepLR(optimizer,10,gamma=0.5)
        return [optimizer],[scheduler]
   
    def training_step(self,batch,batch_idx):
        _, _, answers, *batch_input = batch
        logits = self(*batch_input)
        loss = self.criterion(logits, answers)
        acc = self.train_acc(logits,answers)
        self.log('step_loss',loss,prog_bar = True,logger=False)
        return {'loss': loss,'n_samples':len(answers)}
    
    def training_epoch_end(self, training_step_outputs):
        loss=0
        n_samples = 0
        for step_out in training_step_outputs:
            loss += step_out['loss']
            n_samples += step_out['n_samples']
        loss = loss/n_samples
        self.log('train_loss',loss,logger=True)
        self.log('train_acc',self.train_acc.compute(),logger=True)
        
    def validation_step(self,batch,batch_idx):
        _, _, answers, *batch_input = batch
        logits = self(*batch_input)
        loss = self.criterion(logits, answers)
        acc = self.valid_acc(logits,answers)
        return {'val_loss': loss,'n_samples':len(answers)}
    
    def validation_epoch_end(self, val_step_outputs):
        loss=0
        n_samples = 0
        for step_out in val_step_outputs:
            loss += step_out['val_loss']
            n_samples += step_out['n_samples']
        loss = loss/n_samples
        val_acc = self.valid_acc.compute()
        self.log('val_acc',val_acc,prog_bar = True,logger=True)
        self.log('val_loss',loss,logger=True)
        
    def test_step(self,batch,batch_idx):
        _, _, answers, *batch_input = batch
        logits = self(*batch_input)
        acc = self.test_acc(logits,answers)
    
    def test_epoch_end(self,test_step_outputs):
        test_acc = self.test_acc.compute()
        print(f"The test accuracy is {test_acc}")
        self.log('test_acc',test_acc,logger=True)
        
        
    

## Recreating papers results on MSVD-QA, MSRVTT-QA, TGIF-QA_FrameQA

### MSVD-QA

In [10]:
max_epochs = 1

model_kwargs = {
        'question_type': msvd_glove_data_module.question_type,
        'vision_dim': 2048,
        'module_dim': 512,
        'word_dim': 300,
        'k_max_frame_level': 16,
        'k_max_clip_level': 8,
        'spl_resolution': 1,
        'vocab': msvd_glove_data_module.vocab
    }
model = HCRN_glove(
    glove_matrix=msvd_glove_data_module.glove_matrix,
    lr=0.0001,
    model_kwargs=model_kwargs,
    optimizer='Adam'
)

checkpoint_callback_val_acc = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    dirpath='models_checkpoints/msvd/baseline',
    filename='msvd-base-{epoch:02d}-{val_acc:.2f}',
    save_top_k=1,
    mode='max',
)
wandb_logger = WandbLogger(name='HCRN-MSVD-base',project='video-qa-hcrn-recvis')

trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    logger = wandb_logger,
    callbacks =[checkpoint_callback_val_acc]
)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Checkpoint directory models_checkpoints/msvd/baseline exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [11]:
trainer.fit(model,msvd_glove_data_module)


  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | train_acc | Accuracy         | 0     
2 | valid_acc | Accuracy         | 0     
3 | test_acc  | Accuracy         | 0     
4 | model     | HCRNNetwork      | 43.7 M
-----------------------------------------------
43.7 M    Trainable params
0         Non-trainable params
43.7 M    Total params


loading questions from data/msvd-qa/glove_question_embedding/msvd-qa_val_questions.pt
loading appearance feature from data/msvd-qa/msvd-qa_appearance_feat.h5
loading motion feature from data/msvd-qa/msvd-qa_motion_feat.h5


loading questions from data/msvd-qa/glove_question_embedding/msvd-qa_train_questions.pt
loading appearance feature from data/msvd-qa/msvd-qa_appearance_feat.h5
loading motion feature from data/msvd-qa/msvd-qa_motion_feat.h5


1

In [16]:
test_res = trainer.test(verbose=False)

loading questions from data/msvd-qa/glove_question_embedding/msvd-qa_test_questions.pt
loading appearance feature from data/msvd-qa/msvd-qa_appearance_feat.h5
loading motion feature from data/msvd-qa/msvd-qa_motion_feat.h5


The test accuracy is 0.32553014159202576



In [18]:
wandb.log({'test_acc':test_res[0]['test_acc']})

In [13]:
wandb.save(trainer.checkpoint_callback.best_model_path)

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


['/home/jupyter/video-qa-recvis/wandb/run-20210102_013312-2yuvqn87/files/msvd-base-epoch=00-val_acc=0.31-v1.ckpt']

### MSRVTT-QA

In [ ]:
max_epochs = 25

model_kwargs = {
        'question_type': msrvtt_glove_data_module.question_type,
        'vision_dim': 2048,
        'module_dim': 512,
        'word_dim': 300,
        'k_max_frame_level': 16,
        'k_max_clip_level': 8,
        'spl_resolution': 1,
        'vocab': msrvtt_glove_data_module.vocab
    }
model = HCRN_glove(
    glove_matrix=msrvtt_glove_data_module.glove_matrix,
    lr=0.0001, 
    model_kwargs=model_kwargs, 
    optimizer='Adam'
)

checkpoint_callback_val_acc = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    dirpath='models_checkpoints/msrvtt/baseline',
    filename='msrvtt-base-{epoch:02d}-{val_acc:.2f}',
    save_top_k=1,
    mode='max',
)
wandb_logger = WandbLogger(name='HCRN-MSRVTT-base',project='video-qa-hcrn-recvis')

trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    logger = wandb_logger,
    callbacks =[checkpoint_callback_val_acc]
)

In [ ]:
trainer.fit(model,msrvtt_glove_data_module)

In [ ]:
test_res = trainer.test(verbose=False)
wandb.log({'test_acc':test_res[0]['test_acc']})

In [ ]:
wandb.save(trainer.checkpoint_callback.best_model_path)

### TGIF-QA FrameQA

In [ ]:
max_epochs = 25

model_kwargs = {
        'question_type': tgif_glove_data_module.question_type,
        'vision_dim': 2048,
        'module_dim': 512,
        'word_dim': 300,
        'k_max_frame_level': 16,
        'k_max_clip_level': 8,
        'spl_resolution': 1,
        'vocab': tgif_glove_data_module.vocab
    }
model = HCRN_glove(
    glove_matrix=tgif_glove_data_module.glove_matrix,
    lr=0.0001, 
    model_kwargs=model_kwargs, 
    optimizer='Adam'
)

checkpoint_callback_val_acc = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    dirpath='models_checkpoints/tgif-qa_frameqa/baseline',
    filename='tgif-qa_frameqa-base-{epoch:02d}-{val_acc:.2f}',
    save_top_k=1,
    mode='max',
)
wandb_logger = WandbLogger(name='HCRN-MSRVTT-base',project='video-qa-hcrn-recvis')

trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    logger = wandb_logger,
    callbacks =[checkpoint_callback_val_acc]
)

In [ ]:
trainer.fit(model,tgif_glove_data_module)

In [ ]:
test_res = trainer.test(verbose=False)
wandb.log({'test_acc':test_res[0]['test_acc']})

In [ ]:
wandb.save(trainer.checkpoint_callback.best_model_path)

## Improving using AdamW

### MSVD-QA

In [ ]:
max_epochs = 25

model_kwargs = {
        'question_type': msvd_glove_data_module.question_type,
        'vision_dim': 2048,
        'module_dim': 512,
        'word_dim': 300,
        'k_max_frame_level': 16,
        'k_max_clip_level': 8,
        'spl_resolution': 1,
        'vocab': msvd_glove_data_module.vocab
    }
model = HCRN_glove(
    glove_matrix=msvd_glove_data_module.glove_matrix,
    lr=0.0001,
    model_kwargs=model_kwargs,
    optimizer='AdamW'
)

checkpoint_callback_val_acc = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    dirpath='models_checkpoints/msvd/baseline',
    filename='msvd-adamw-{epoch:02d}-{val_acc:.2f}',
    save_top_k=1,
    mode='max',
)
wandb_logger = WandbLogger(name='HCRN-MSVD-base',project='video-qa-hcrn-recvis')

trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    logger = wandb_logger,
    callbacks =[checkpoint_callback_val_acc]
)

In [ ]:
trainer.fit(model,msvd_glove_data_module)

In [ ]:
test_res = trainer.test(verbose=False)
wandb.log({'test_acc':test_res[0]['test_acc']})

In [ ]:
wandb.save(trainer.checkpoint_callback.best_model_path)

### MSRVTT-QA

In [9]:
max_epochs = 25

model_kwargs = {
        'question_type': msrvtt_glove_data_module.question_type,
        'vision_dim': 2048,
        'module_dim': 512,
        'word_dim': 300,
        'k_max_frame_level': 16,
        'k_max_clip_level': 8,
        'spl_resolution': 1,
        'vocab': msrvtt_glove_data_module.vocab
    }
model = HCRN_glove(
    glove_matrix=msrvtt_glove_data_module.glove_matrix,
    lr=0.0001, 
    model_kwargs=model_kwargs, 
    optimizer='AdamW'
)

checkpoint_callback_val_acc = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    dirpath='models_checkpoints/msrvtt/baseline',
    filename='msrvtt-adamw-{epoch:02d}-{val_acc:.2f}',
    save_top_k=1,
    mode='max',
)
wandb_logger = WandbLogger(name='HCRN-MSRVTT-base',project='video-qa-hcrn-recvis')

trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    logger = wandb_logger,
    callbacks =[checkpoint_callback_val_acc]
)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Checkpoint directory models_checkpoints/msrvtt/baseline exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model,msrvtt_glove_data_module)


  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | train_acc | Accuracy         | 0     
2 | valid_acc | Accuracy         | 0     
3 | test_acc  | Accuracy         | 0     
4 | model     | HCRNNetwork      | 48.5 M
-----------------------------------------------
48.5 M    Trainable params
0         Non-trainable params
48.5 M    Total params


loading questions from data/msrvtt-qa/glove_question_embedding/msrvtt-qa_val_questions.pt
loading appearance feature from data/msrvtt-qa/msrvtt-qa_appearance_feat.h5
loading motion feature from data/msrvtt-qa/msrvtt-qa_motion_feat.h5


loading questions from data/msrvtt-qa/glove_question_embedding/msrvtt-qa_train_questions.pt
loading appearance feature from data/msrvtt-qa/msrvtt-qa_appearance_feat.h5
loading motion feature from data/msrvtt-qa/msrvtt-qa_motion_feat.h5


In [ ]:
test_res = trainer.test(verbose=False)
wandb.log({'test_acc':test_res[0]['test_acc']})

In [ ]:
wandb.save(trainer.checkpoint_callback.best_model_path)

### TGIF-QA FrameQA

In [ ]:
max_epochs = 25

model_kwargs = {
        'question_type': tgif_glove_data_module.question_type,
        'vision_dim': 2048,
        'module_dim': 512,
        'word_dim': 300,
        'k_max_frame_level': 16,
        'k_max_clip_level': 8,
        'spl_resolution': 1,
        'vocab': tgif_glove_data_module.vocab
    }
model = HCRN_glove(
    glove_matrix=tgif_glove_data_module.glove_matrix,
    lr=0.0001, 
    model_kwargs=model_kwargs, 
    optimizer='AdamW'
)

checkpoint_callback_val_acc = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    dirpath='models_checkpoints/tgif-qa_frameqa/baseline',
    filename='tgif-qa_frameqa-base-{epoch:02d}-{val_acc:.2f}',
    save_top_k=1,
    mode='max',
)
wandb_logger = WandbLogger(name='HCRN-MSRVTT-base',project='video-qa-hcrn-recvis')

trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    logger = wandb_logger,
    callbacks =[checkpoint_callback_val_acc]
)

In [ ]:
trainer.fit(model,tgif_glove_data_module)

In [ ]:
test_res = trainer.test(verbose=False)
wandb.log({'test_acc':test_res[0]['test_acc']})

In [ ]:
wandb.save(trainer.checkpoint_callback.best_model_path)